In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
def getFileName(ltr, num, tst) :
    div = os.sep
    base_path = os.getcwd() + div + "HW4data"
    path = base_path+div+ ltr + div + num + div
    return path + 'Train'*(tst==0) + 'Test'*(tst==1) + 'X'*(ltr is 'b' and tst == 0) + "_" + num + ltr + '.csv'


In [ ]:

scoress=[]

div = os.sep
path = os.getcwd() + div + "HW4data" + div + "hw4_result.csv"
hw4_result = pd.read_csv(path)

base = ["a","b"]
ext = ["1", "2", "3", "4", "5"]
for letter in base :
    for number in ext :
        

        trainer = pd.read_csv(getFileName(letter, number, 0))
        tester = pd.read_csv(getFileName(letter, number, 1))

        features = tester.columns.tolist()
        x = trainer.loc[:, features].values
        y = trainer.loc[:,['Output']].values
        x = StandardScaler().fit_transform(x)
        train = pd.DataFrame(data=x, columns=tester.columns.tolist())
        train = train.join(trainer['Output'])


        from sklearn.decomposition import PCA
        pca = PCA(n_components=5)
        principalComponents = pca.fit_transform(x)
        principalDf = pd.DataFrame(data = principalComponents
                     , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5'])
        inverse_df = pca.inverse_transform(principalDf)
        MSE = ((x-inverse_df)**2).sum(axis=1)
        n=10
        MSE_max_scores = pd.Series(MSE).nlargest(n).index
        
        
        lst = []
        for i in MSE_max_scores :
            if (MSE[int(i)]) > 4 :
                lst.append(i)
        train.drop(train.index[lst])

        
        predict = np.zeros(len(train))
        models = []
        scores = []
        X = np.arange(len(train))
        ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
        for train_index,test_index in ss.split(X):
            xtr, ytr = train[features].iloc[train_index], train['Output'].iloc[train_index]
            xvl, yvl = train[features].iloc[test_index], train['Output'].iloc[test_index]
            regressor = LinearRegression()
            regressor.fit(xtr, ytr)
            models.append(regressor)
            predict[test_index]=(regressor.predict(xvl))
            
        
        score = np.sqrt(metrics.mean_squared_error(train['Output'], np.clip(predict, a_min=0, a_max=None)))
        
        test = pd.read_csv(getFileName(letter, number, 1))
        
        out = []
        temp = []
        for model in models :
            temp.append(model.predict(test))
        out.append(np.mean(temp, axis=0))
        
        
        print('Predicted score is :', score)
        scoress.append(score)
        string = number + letter
        for i in range(len(out[0])) :
            hw4_result[string][i]=out[0][i]
        
display(hw4_result)

In [ ]:
hw4_result.to_csv('Gottlieb_hw4_result.csv', index=False)